## Import lib

In [ ]:
import cv2 as cv
import os
import matplotlib.pyplot as plt
import numpy as np
from feature import *

### Load image file

In [ ]:
image_path = "D:\\Hoc\\Mutimedia database system\\System\\Multimedia-Database-System\\Banana (10).jpg"
image = cv.imread(image_path)

In [ ]:
def Centers(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()
        centers = []
        for line in lines:
            center = [float(x) for x in line.strip().split()]
            centers.append(center)
        return centers

def euclidean_distance(vector1, vector2):
    # Tính khoảng cách Euclidean giữa hai vector
    return np.linalg.norm(vector1 - vector2)

def find_Center(point, centers):
    min_distance = float('inf')
    closest_center = None
    closest_index = -1
    
    for i,center in enumerate(centers):
        
        distance = euclidean_distance(point, center)
        
        if distance < min_distance:
            min_distance = distance
            closest_center = center
            closest_index = i
    
    return closest_center, closest_index


def find_cluster(point):
    folder_path = ""
    filename ="cluster_centers.txt"
    file_path = os.path.join(folder_path, filename)
    
    while os.path.exists(file_path):
        centers = Centers(file_path)
        center, index = find_Center(point, centers)
        
        folder_path = folder_path+ "/"+ str(index.item())
        file_path = os.path.join(folder_path, filename)
    
    return folder_path

In [ ]:
def cosin_distance(vector1, vector2):
    # Tính khoảng cách cosin giữa hai vector
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

In [ ]:
image_HSV = average_HSV(image)

cluster_path = find_cluster(image_HSV)

image_feature = get_image_feature(image,cluster_path)

metadata_path = os.path.join(cluster_path, "metadata.txt")

metadata = {}

with open(metadata_path,"r") as file:
    for line in file:
        key, value = line.strip().split(": ",1)
        key = key.strip()
        value = value.strip()
        
        dict_value = eval(value)
        
        for key1,value1 in dict_value.items():
            if isinstance(value1, list):
                pass
            else:
                try:
                    dict_value[key1] = float(value1)
                except ValueError:
                    pass
                
        metadata[key] = dict_value


for key, value in metadata:
    dict_tmp = metadata[key]
    feature = dict_tmp['Bow']
    distance = cosin_distance(image_feature, feature)
    metadata[key]=distance

sorted_metadata = sorted(metadata.items(), key=lambda x: x[1])[:3]

fig, axes = plt.subplots(1, 3)

for i,(path,_) in enumerate(sorted_metadata):
    image = cv.imread(path)
    image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    axes[i].imshow(image_rgb)
    axes[i].axis('off')
    
plt.show()
